In [1]:
import torch
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.datasets import fetch_20newsgroups

In [2]:
import os
os.chdir("..")

### Read data and train the model

In [3]:
categories = ["comp.graphics","sci.space","rec.sport.baseball"]
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

print('total texts in train:',len(newsgroups_train.data))
print('total texts in test:',len(newsgroups_test.data))

total texts in train: 1774
total texts in test: 1180


In [4]:
# newsgroups_train.target
# newsgroups_train.target_names
newsgroups_test.target_names

['comp.graphics', 'rec.sport.baseball', 'sci.space']

In [5]:
# Each one is the list of strings
data_train, data_test = newsgroups_train.data, newsgroups_test.data

In [6]:
print(total_words)

NameError: name 'total_words' is not defined

In [12]:
vocab = Counter()

for text in newsgroups_train.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1
        
for text in newsgroups_test.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1
        
total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    index2word = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i
        index2word[i] = word.lower()
        
    return word2index, index2word

word2index, index2word = get_word_2_index(vocab)

In [13]:
def convert2bow(texts):
    bow = np.zeros((len(texts), total_words))
    for text_idx, text in enumerate(texts):
        for word in text.split(' '):
            bow[text_idx, word2index[word.lower()]] += 1
    return bow

X_train, X_test = torch.tensor(convert2bow(data_train), dtype=torch.float32), torch.tensor(convert2bow(data_test), dtype=torch.float32)
y_train, y_test = newsgroups_train.target, newsgroups_test.target
y_train_names, y_test_names = newsgroups_train.target_names, newsgroups_test.target_names

In [14]:
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.data[i*batch_size:i*batch_size+batch_size]
    categories = df.target[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        layer = np.zeros(total_words,dtype=float)
        for word in text.split(' '):
            layer[word2index[word.lower()]] += 1
            
        batches.append(layer)
        
    for category in categories:
        index_y = -1
        if category == 0:
            index_y = 0
        elif category == 1:
            index_y = 1
        else:
            index_y = 2
        results.append(index_y)
            
     
    return np.array(batches),np.array(results)

In [15]:
# Parameters
learning_rate = 0.01
num_epochs = 10
batch_size = 150
display_step = 1

# Network Parameters
hidden_size = 100      # 1st layer and 2nd layer number of features
input_size = total_words # Words in vocab
num_classes = 3         # Categories: graphics, sci.space and baseball

In [16]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class OurNet(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
        super(OurNet, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)
 
     def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [18]:
# input [batch_size, n_labels]
# output [max index for each item in batch, ... ,batch_size-1]
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(2, 5), requires_grad=True)
print(">>> batch of size 2 and 5 possible classes")
print(input)
target = Variable(torch.LongTensor(2).random_(5))
print(">>> array of size 'batch_size' with the index of the maxium label for each item")
print(target)
output = loss(input, target)
output.backward()

>>> batch of size 2 and 5 possible classes
tensor([[ 0.2462, -1.0717, -1.9975,  0.3517, -0.2022],
        [-0.2601, -0.9926,  2.2672, -1.2531, -1.4651]], requires_grad=True)
>>> array of size 'batch_size' with the index of the maxium label for each item
tensor([0, 4])


In [19]:
net = OurNet(input_size, hidden_size, num_classes)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

# Train the Model
for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        #print("articles",articles)
        #print(batch_x, labels)
        #print("size labels",labels.size())
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.item()))

Epoch [1/10], Step [4/11], Loss: 1.2979
Epoch [1/10], Step [8/11], Loss: 0.4154
Epoch [2/10], Step [4/11], Loss: 0.0927
Epoch [2/10], Step [8/11], Loss: 0.2134
Epoch [3/10], Step [4/11], Loss: 0.0609
Epoch [3/10], Step [8/11], Loss: 0.0002
Epoch [4/10], Step [4/11], Loss: 0.0000
Epoch [4/10], Step [8/11], Loss: 0.0000
Epoch [5/10], Step [4/11], Loss: 0.0008
Epoch [5/10], Step [8/11], Loss: 0.0000
Epoch [6/10], Step [4/11], Loss: 0.0000
Epoch [6/10], Step [8/11], Loss: 0.0968
Epoch [7/10], Step [4/11], Loss: 0.0000
Epoch [7/10], Step [8/11], Loss: 0.0000
Epoch [8/10], Step [4/11], Loss: 0.0000
Epoch [8/10], Step [8/11], Loss: 0.0001
Epoch [9/10], Step [4/11], Loss: 0.0002
Epoch [9/10], Step [8/11], Loss: 0.0000
Epoch [10/10], Step [4/11], Loss: 0.0003
Epoch [10/10], Step [8/11], Loss: 0.0001


In [20]:
# Test the Model
correct = 0
total = 0
total_test_data = len(newsgroups_test.target)
batch_x_test,batch_y_test = get_batch(newsgroups_test,0,total_test_data)
articles = Variable(torch.FloatTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)
outputs = net(articles)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()
    
print('Accuracy of the network on the 1180 test articles: %d %%' % (100 * correct / total))

Accuracy of the network on the 1180 test articles: 90 %


### Generate set of counterfactuals

In [21]:
from mlexplain.sce.util import choose_k_top_elements_softmax

In [22]:
# Choose 10 objects and display them
X_class_0 = choose_k_top_elements_softmax(net, X_train, K=10, target_class=0)

In [23]:
from mlexplain.sce.text_sce import TextSCE

### Experiment 1. LR=0.1, lambda_coef=0, mu_coef=0

In [24]:
sce = TextSCE(net, target_class=0, index2word=index2word)

In [25]:
sce.fit(X_class_0, lambda_coef=0.0, mu_coef=0.0, n_iter=30, verbose_every_iterations=5, lr=0.1, force_masks_init=False)

[0/30] Cost: 318.3843688964844 [318.3843688964844, 0.0, 0.0]
[5/30] Cost: 282.36541748046875 [282.36541748046875, 0.0, 0.0]
[10/30] Cost: 235.9352264404297 [235.9352264404297, 0.0, 0.0]
[15/30] Cost: 187.18814086914062 [187.18814086914062, 0.0, 0.0]
[20/30] Cost: 140.813232421875 [140.813232421875, 0.0, 0.0]
[25/30] Cost: 90.9652328491211 [90.9652328491211, 0.0, 0.0]


In [26]:
sce.top_k_words_all_masks(k=5)

[['computer', 'package', 'file', 'card', 'graphics'],
 ['computer', 'package', 'card', 'file', 'pc'],
 ['computer', 'package', 'file', 'card', 'pc'],
 ['computer', 'package', 'card', 'file', 'pc'],
 ['computer', 'package', 'card', 'file', 'pc'],
 ['computer', 'package', 'card', 'file', 'anybody'],
 ['computer', 'package', 'file', 'pc', 'algorithm'],
 ['computer', 'package', 'card', 'pc', 'algorithm'],
 ['computer', 'package', 'pc', 'card', 'file'],
 ['computer', 'algorithm', 'pc', 'package', 'address.']]

In [27]:
sce.top_k_words()

['computer',
 'package',
 'file',
 'card',
 'pc',
 'algorithm',
 'polygon',
 'graphics',
 'anybody',
 'wanted:']

### Experiment 2. LR=0.1, lambda_coef=0.05, mu_coef=1.0

In [28]:
sce2 = TextSCE(net, target_class=0, index2word=index2word)

In [29]:
sce2.fit(X_class_0, lambda_coef=0.05, mu_coef=1.0, n_iter=30, verbose_every_iterations=5, lr=0.1, force_masks_init=False)

[0/30] Cost: 318.3843688964844 [318.3843688964844, 0.0, 0.0]
[5/30] Cost: 301.8505554199219 [293.4259033203125, 8.197685241699219, 0.2269568145275116]
[10/30] Cost: 283.6103210449219 [266.95635986328125, 16.152647018432617, 0.5013245344161987]
[15/30] Cost: 258.5602111816406 [233.5675811767578, 24.107776641845703, 0.88484787940979]
[20/30] Cost: 234.97317504882812 [200.9049530029297, 32.87041473388672, 1.197807788848877]
[25/30] Cost: 212.79661560058594 [170.45352172851562, 40.885311126708984, 1.4577829837799072]


In [30]:
sce2.top_k_words_all_masks(k=5)

[['computer', 'package', 'file', 'card', 'graphics'],
 ['computer', 'package', 'file', 'card', 'pc'],
 ['computer', 'package', 'file', 'card', 'pc'],
 ['computer', 'package', 'card', 'file', 'pc'],
 ['computer', 'package', 'card', 'file', 'pc'],
 ['computer', 'package', 'card', 'file', 'anybody'],
 ['computer', 'package', 'file', 'card', 'pc'],
 ['computer', 'package', 'card', 'pc', 'file'],
 ['computer', 'package', 'card', 'file', 'pc'],
 ['computer', 'package', 'pc', 'card', 'file']]

In [31]:
sce2.top_k_words(k=10)

['computer',
 'package',
 'card',
 'file',
 'pc',
 'algorithm',
 'polygon',
 'anybody',
 'graphics',
 'polygons']

### Experiment 3. Top-10 words to remove. LR=0.1, lambda_coef=0.05, mu_coef=1.0

In [32]:
K = 10
sces = []
for target_class in range(3):
    print(f"Processing class {target_class}")
    X_class_target = choose_k_top_elements_softmax(net, X_train, K=10, target_class=target_class)
    sce = TextSCE(net, target_class=target_class, index2word=index2word)
    sce.fit(X_class_target, lambda_coef=0.05, mu_coef=1.0, n_iter=30, verbose_every_iterations=10, lr=0.1, force_masks_init=False)
    sces.append(sce)

Processing class 0
[0/30] Cost: 318.3843688964844 [318.3843688964844, 0.0, 0.0]
[10/30] Cost: 283.6103210449219 [266.95635986328125, 16.152647018432617, 0.5013245344161987]
[20/30] Cost: 234.97317504882812 [200.9049530029297, 32.87041473388672, 1.197807788848877]
Processing class 1
[0/30] Cost: 89.43406677246094 [89.43406677246094, 0.0, 0.0]
[10/30] Cost: 27.00507354736328 [2.5235981941223145, 23.599483489990234, 0.881991982460022]
[20/30] Cost: -0.7504318952560425 [-37.98480987548828, 35.86570358276367, 1.368674397468567]
Processing class 2
[0/30] Cost: 279.2134704589844 [279.2134704589844, 0.0, 0.0]
[10/30] Cost: 210.59249877929688 [185.0362091064453, 24.91416358947754, 0.6421241760253906]
[20/30] Cost: 147.4491424560547 [100.28791046142578, 46.16477966308594, 0.9964428544044495]


In [33]:
for target_class, sce in enumerate(sces):
    print(f"Target class: {target_class}. Top words:")
    print(sce.top_k_words(k=10))

Target class: 0. Top words:
['computer', 'package', 'card', 'file', 'pc', 'algorithm', 'polygon', 'anybody', 'graphics', 'polygons']
Target class: 1. Top words:
['baseball', 'players', 'season', 'stats', 'team', 'rockies', 'steve', 'home', 'mike', 'pitcher']
Target class: 2. Top words:
['observatory', 'orbit', 'planet', 'sci\nlines:', 'university]\noriginal-sender:', '[via', 'isu@vacation.venari.cs.cmu.edu\ndistribution:', 'planets', '(713)', 'astronomy']


### 

In [34]:
for idx, x in enumerate(data_train):
    if 'sci\nlines' in x.lower() and 'prb@access.digex.com' in x.lower():
        print(idx, x)

1563 From: dpage@ra.csc.ti.com (Doug Page)
Subject: Re: Sr-71 in propoganda films?
Nntp-Posting-Host: ra
Organization: Texas Instruments
Distribution: sci
Lines: 28

In article <1993Apr5.220610.1532@sequent.com>, bigfoot@sequent.com (Gregory Smith) writes:
|> mccall@mksol.dseg.ti.com (fred j mccall 575-3539) writes:
|> 
|> >In <1phv98$jbk@access.digex.net> prb@access.digex.com (Pat) writes:
|> 
|> 
|> >>THe SR-71 stopped being a real secret by the mid 70's.
|> >>I had a friend in high school who had a poster with it's picture.
|> 
|> >It was known well before that.  I built a model of it sometime in the
|> >mid 60's, billed as YF-12A/SR-71.  The model was based on YF-12A specs
|> >and had a big radar in the nose and 8 AAMs in closed bays on the
|> >underside of the fuselage.  The description, even then, read "speeds
|> >in excess of Mach 3 at altitudes exceeding 80,000 feet."
|> 
|> L.B.J. publically announced the existance of the Blackbird program
|> in 1964.


He's also the one who d

In [35]:
print(data_train[1563])

From: dpage@ra.csc.ti.com (Doug Page)
Subject: Re: Sr-71 in propoganda films?
Nntp-Posting-Host: ra
Organization: Texas Instruments
Distribution: sci
Lines: 28

In article <1993Apr5.220610.1532@sequent.com>, bigfoot@sequent.com (Gregory Smith) writes:
|> mccall@mksol.dseg.ti.com (fred j mccall 575-3539) writes:
|> 
|> >In <1phv98$jbk@access.digex.net> prb@access.digex.com (Pat) writes:
|> 
|> 
|> >>THe SR-71 stopped being a real secret by the mid 70's.
|> >>I had a friend in high school who had a poster with it's picture.
|> 
|> >It was known well before that.  I built a model of it sometime in the
|> >mid 60's, billed as YF-12A/SR-71.  The model was based on YF-12A specs
|> >and had a big radar in the nose and 8 AAMs in closed bays on the
|> >underside of the fuselage.  The description, even then, read "speeds
|> >in excess of Mach 3 at altitudes exceeding 80,000 feet."
|> 
|> L.B.J. publically announced the existance of the Blackbird program
|> in 1964.


He's also the one who dubbed

#### Experiment 4 (robustness for 'comp.graphics')

In [36]:
K = 10
target_class = 0

X_class_target = choose_k_top_elements_softmax(net, X_train, K=10, target_class=target_class)

sce_0 = TextSCE(net, target_class=target_class, index2word=index2word)
sce_1 = TextSCE(net, target_class=target_class, index2word=index2word)

In [37]:
sce_0.fit(X_class_target, lambda_coef=0, mu_coef=0, n_iter=30, verbose_every_iterations=10, lr=0.1, force_masks_init=False)

[0/30] Cost: 318.3843688964844 [318.3843688964844, 0.0, 0.0]
[10/30] Cost: 235.9352264404297 [235.9352264404297, 0.0, 0.0]
[20/30] Cost: 140.813232421875 [140.813232421875, 0.0, 0.0]


In [49]:
sce_1.fit(X_class_target, lambda_coef=0.05, mu_coef=50.0, n_iter=30, verbose_every_iterations=10, lr=0.1, force_masks_init=False)

[0/30] Cost: 201.779541015625 [-9.46551513671875, 87.28343963623047, 123.96162414550781]
[10/30] Cost: 93.113525390625 [-48.03938293457031, 108.80816650390625, 32.3447380065918]
[20/30] Cost: 50.35553741455078 [-107.24150085449219, 125.9161376953125, 31.6809024810791]


In [58]:
sce_0.top_k_words_all_masks(k=7)

[['computer', 'package', 'file', 'card', 'graphics', 'pc', 'polygon'],
 ['computer', 'package', 'card', 'file', 'pc', 'algorithm', 'graphics'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'graphics'],
 ['computer', 'package', 'card', 'file', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'card', 'file', 'pc', '|\n', 'packages'],
 ['computer', 'package', 'card', 'file', 'anybody', 'pc', '|\n'],
 ['computer', 'package', 'file', 'pc', 'algorithm', 'card', 'polygon'],
 ['computer', 'package', 'card', 'pc', 'algorithm', 'file', 'polygon'],
 ['computer', 'package', 'pc', 'card', 'file', 'address.', 'algorithm'],
 ['computer', 'algorithm', 'pc', 'package', 'address.', 'file', 'card']]

In [57]:
sce_1.top_k_words_all_masks(k=7)

[['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon'],
 ['computer', 'package', 'file', 'card', 'pc', 'algorithm', 'polygon']]